In [1]:
!gdown 1TW77KJvx3pCV3kiVJrRvita5yEq6Q_op

Downloading...
From: https://drive.google.com/uc?id=1TW77KJvx3pCV3kiVJrRvita5yEq6Q_op
To: /content/anon_processed_df_parquet
100% 21.3M/21.3M [00:00<00:00, 63.0MB/s]


In [2]:
import pandas as pd

df_base = pd.read_parquet("anon_processed_df_parquet")

In [3]:
df_base.describe()

,steps_awake_mean,sleep_asleep_weekday_mean,sleep_asleep_weekend_mean,sleep_in_bed_weekday_mean,sleep_in_bed_weekend_mean,sleep_ratio_asleep_in_bed_weekday_mean,sleep_ratio_asleep_in_bed_weekend_mean,sleep_in_bed_iqr,sleep_asleep_iqr,sleep_ratio_asleep_in_bed_iqr,...,life_stress,med_nonmed_dnu,life_activity_eating,life_red_stop_alcoh,comorbid_neuropathic,comorbid_arthritis,phq9_score_start,phq9_score_end,phq9_cat_end,phq9_cat_start
count,35674.000000,35329.000000,34301.000000,35329.000000,34301.000000,34123.000000,32260.000000,22228.000000,22228.000000,15532.000000,...,35694.000000,35694.000000,35694.000000,35694.000000,35694.000000,35694.000000,10866.000000,10866.000000,10866.000000,10866.000000
mean,8471.100429,364.581904,397.398778,412.144136,449.207385,0.887317,0.886492,115.704888,103.586243,0.040923,...,0.039278,0.489634,0.183476,0.040679,0.302012,0.297753,7.661881,7.467605,1.116602,1.155255
std,4267.165345,128.895531,162.401119,132.488135,170.465514,0.129016,0.132958,84.720396,75.534577,0.032043,...,0.194259,0.499900,0.387062,0.197548,0.579062,0.512623,6.028646,6.037733,1.159894,1.168189
min,49.285714,0.000000,0.000000,0.000000,0.000000,0.088608,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5605.931319,310.916667,312.500000,380.916667,393.000000,0.909928,0.902172,64.250000,58.750000,0.022577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,0.000000,0.000000
50%,7847.952381,397.333333,432.500000,441.916667,483.000000,0.933812,0.933658,95.250000,85.250000,0.031693,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,6.000000,1.000000,1.000000
75%,10501.750000,448.250000,503.500000,487.750000,550.000000,0.951378,0.955775,138.750000,123.250000,0.045741,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,11.000000,11.000000,2.000000,2.000000
max,52048.928571,1085.000000,1250.000000,1085.000000,1400.000000,1.000000,1.000000,764.750000,714.750000,0.511468,...,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,27.000000,27.000000,4.000000,4.000000


In [4]:
df_base["user_id"] = df_base.index.str.split("_").str[0].astype(int)
df_base["month"] = df_base.index.str.split("_").str[1].astype(int)

In [5]:
user_month_set = set()
for user_id, month in zip(df_base["user_id"], df_base["month"]):
    user_month_set.add((user_id, month))

user_trimonth_set = set()
for user_id, month in user_month_set:
    if month % 3 == 1 and (user_id, month + 1) in user_month_set and (user_id, month + 2) in user_month_set:
        user_trimonth_set.add(f"{user_id}_{month}")

In [6]:
df_base_copy = df_base.copy()

df_readier = df_base_copy[df_base_copy.index.isin(user_trimonth_set)].copy()

df_base_copy["month"] -= 2
df_base_copy.index = df_base_copy.index.str.split("_").str[0] + "_" + df_base_copy["month"].astype(str)
for col in df_base_copy.columns:
    if col in ["phq9_score_start", "phq9_cat_start", "phq9_score_end"]:
        col_name = col
        df_base_copy.rename(columns={col: f"{col_name}_3"}, inplace=True)
        df_readier[f"{col_name}_3"] = df_base_copy[df_base_copy.index.isin(user_trimonth_set)][f"{col_name}_3"]

In [7]:
df_readier

,steps_awake_mean,sleep_asleep_weekday_mean,sleep_asleep_weekend_mean,sleep_in_bed_weekday_mean,sleep_in_bed_weekend_mean,sleep_ratio_asleep_in_bed_weekday_mean,sleep_ratio_asleep_in_bed_weekend_mean,sleep_in_bed_iqr,sleep_asleep_iqr,sleep_ratio_asleep_in_bed_iqr,...,comorbid_arthritis,phq9_score_start,phq9_score_end,phq9_cat_end,phq9_cat_start,user_id,month,phq9_score_start_3,phq9_score_end_3,phq9_cat_start_3
4938_1,10615.285714,474.333333,467.0,520.166667,518.5,0.912700,0.903756,90.75,65.00,0.048269,...,0.0,NaN,NaN,NaN,NaN,4938,1,6.0,8.0,1.0
4938_4,10509.857143,401.333333,510.0,432.250000,548.0,0.929071,0.931062,53.00,60.25,0.023311,...,0.0,NaN,NaN,NaN,NaN,4938,4,8.0,8.0,1.0
4938_7,10083.214286,510.750000,562.0,548.000000,601.0,0.931547,0.935708,131.50,113.25,0.022707,...,0.0,NaN,NaN,NaN,NaN,4938,7,8.0,4.0,1.0
3579_1,1149.500000,554.416667,709.0,584.250000,767.0,0.949565,0.928763,172.00,172.50,0.030802,...,0.0,NaN,NaN,NaN,NaN,3579,1,10.0,3.0,2.0
3579_4,3881.642857,626.750000,614.0,660.000000,648.5,0.948991,0.946753,218.00,205.50,NaN,...,0.0,NaN,NaN,NaN,NaN,3579,4,3.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758_1,13113.785714,451.166667,436.5,490.250000,478.5,0.920491,0.910999,146.00,146.25,0.023238,...,1.0,NaN,NaN,NaN,NaN,3758,1,2.0,5.0,0.0
3758_4,5716.928571,480.083333,541.5,531.083333,601.5,0.906814,0.902381,182.25,137.50,0.050627,...,1.0,NaN,NaN,NaN,NaN,3758,4,5.0,6.0,1.0
3758_7,8846.428571,27.333333,31.5,29.583333,35.0,0.923944,0.900000,0.00,0.00,NaN,...,1.0,NaN,NaN,NaN,NaN,3758,7,6.0,3.0,1.0
1456_1,5404.500000,377.333333,490.5,396.500000,509.0,0.950864,0.964158,143.75,141.75,NaN,...,2.0,NaN,NaN,NaN,NaN,1456,1,5.0,2.0,1.0


In [8]:
df_readier = df_readier.drop(columns=[
    "user_id", "month",
    "phq9_score_start",
    "phq9_score_end",
    "phq9_cat_start",
    "phq9_cat_end",
    "steps__awake__sum__score",
    "steps__awake__sum__intercept",
    "steps__awake__sum__coeff",
    "steps__mvpa__sum__score",
    "steps__mvpa__sum__intercept",
    "steps__mvpa__sum__coeff",
    "steps__light_activity__sum__score",
    "steps__light_activity__sum__intercept",
    "steps__light_activity__sum__coeff",
    "steps__rolling_6_sum__max__score",
    "steps__rolling_6_sum__max__intercept",
    "steps__rolling_6_sum__max__coeff",
    "steps__streaks__countDistinct__score",
    "steps__streaks__countDistinct__intercept",
    "steps__streaks__countDistinct__coeff",
    "steps__not_moving__sum__score",
    "steps__not_moving__sum__intercept",
    "steps__not_moving__sum__coeff",
    "steps__awake__sum__score_",
    "steps__awake__sum__intercept_",
    "steps__awake__sum__coeff_",
    "steps__mvpa__sum__score_",
    "steps__mvpa__sum__intercept_",
    "steps__mvpa__sum__coeff_",
    "steps__light_activity__sum__score_",
    "steps__light_activity__sum__intercept_",
    "steps__light_activity__sum__coeff_",
    "steps__rolling_6_sum__max__score_",
    "steps__rolling_6_sum__max__intercept_",
    "steps__rolling_6_sum__max__coeff_",
    "steps__streaks__countDistinct__score_",
    "steps__streaks__countDistinct__intercept_",
    "steps__streaks__countDistinct__coeff_",
    "steps__not_moving__sum__score_",
    "steps__not_moving__sum__intercept_",
    "steps__not_moving__sum__coeff_",
    "sleep__main_start_hour_adj__score",
    "sleep__main_start_hour_adj__intercept",
    "sleep__main_start_hour_adj__coeff",
    "sleep__nap_count__score",
    "sleep__nap_count__intercept",
    "sleep__nap_count__coeff",
    "sleep__total_asleep_minutes__score",
    "sleep__total_asleep_minutes__intercept",
    "sleep__total_asleep_minutes__coeff",
    "sleep__main_efficiency__score",
    "sleep__main_efficiency__intercept",
    "sleep__main_efficiency__coeff",
    "sleep__awake__sum__score",
    "sleep__awake__sum__intercept",
    "sleep__awake__sum__coeff",
    "sleep__total_in_bed_minutes__score",
    "sleep__total_in_bed_minutes__intercept",
    "sleep__total_in_bed_minutes__coeff",
    "sleep__awake_regions__countDistinct__score",
    "sleep__awake_regions__countDistinct__intercept",
    "sleep__awake_regions__countDistinct__coeff",
    "sleep_ratio_asleep_in_bed__score",
    "sleep_ratio_asleep_in_bed__intercept",
    "sleep_ratio_asleep_in_bed__coeff",
    "sleep__main_start_hour_adj__score_",
    "sleep__main_start_hour_adj__intercept_",
    "sleep__main_start_hour_adj__coeff_",
    "sleep__nap_count__score_",
    "sleep__nap_count__intercept_",
    "sleep__nap_count__coeff_",
    "sleep__total_asleep_minutes__score_",
    "sleep__total_asleep_minutes__intercept_",
    "sleep__total_asleep_minutes__coeff_",
    "sleep__main_efficiency__score_",
    "sleep__main_efficiency__intercept_",
    "sleep__main_efficiency__coeff_",
    "sleep__awake__sum__score_",
    "sleep__awake__sum__intercept_",
    "sleep__awake__sum__coeff_",
    "sleep__total_in_bed_minutes__score_",
    "sleep__total_in_bed_minutes__intercept_",
    "sleep__total_in_bed_minutes__coeff_",
    "sleep__awake_regions__countDistinct__score_",
    "sleep__awake_regions__countDistinct__intercept_",
    "sleep__awake_regions__countDistinct__coeff_",
    "sleep_ratio_asleep_in_bed__score_",
    "sleep_ratio_asleep_in_bed__intercept_",
    "sleep_ratio_asleep_in_bed__coeff_",
    "med_start",
    "med_stop",
    "med_dose",
    "nonmed_start",
    "nonmed_stop",
    "med_nonmed_dnu",
    "life_meditation",
    "life_stress",
    "life_activity_eating",
    "life_red_stop_alcoh",
])

In [9]:
df_readier.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8494 entries, 4938_1 to 1456_4
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   steps_awake_mean                        8491 non-null   float64
 1   sleep_asleep_weekday_mean               8421 non-null   float64
 2   sleep_asleep_weekend_mean               8260 non-null   float64
 3   sleep_in_bed_weekday_mean               8421 non-null   float64
 4   sleep_in_bed_weekend_mean               8260 non-null   float64
 5   sleep_ratio_asleep_in_bed_weekday_mean  8219 non-null   float64
 6   sleep_ratio_asleep_in_bed_weekend_mean  7898 non-null   float64
 7   sleep_in_bed_iqr                        5996 non-null   float64
 8   sleep_asleep_iqr                        5996 non-null   float64
 9   sleep_ratio_asleep_in_bed_iqr           4330 non-null   float64
 10  steps_mvpa_iqr                          1331 non-null   fl

In [10]:
weird_columns = []

for col in df_readier.columns:
    if df_readier[col].dtype == 'float64' and (df_readier[col] == 0).sum() > 400 and len(df_readier[col].unique()) > 100 and df_readier[col].median() > 100:
      weird_columns.append(col)


In [11]:
for col in weird_columns:
    # replace 0 with NaN
    df_readier[col] = df_readier[col].replace(0, pd.NA)

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_readier, test_size=0.2, random_state=42)

In [13]:
X_train, y_train = df_train.drop(columns=["phq9_score_end_3"]), df_train["phq9_score_end_3"]
X_test, y_test = df_test.drop(columns=["phq9_score_end_3"]), df_test["phq9_score_end_3"]

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

y_train_class = y_train - X_train["phq9_score_start_3"] > 0
y_test_class = y_test - X_test["phq9_score_start_3"] > 0

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean", add_indicator=True)),
    ("scaler", StandardScaler()),
    ("model", RidgeClassifier())
])

pipeline.fit(X_train, y_train_class)
test_pred = pipeline.predict(X_test)

print(classification_report(y_train_class, pipeline.predict(X_train)))
print(classification_report(y_test_class, pipeline.predict(X_test)))

              precision    recall  f1-score   support

       False       0.64      0.89      0.74      4124
        True       0.57      0.22      0.32      2671

    accuracy                           0.63      6795
   macro avg       0.61      0.56      0.53      6795
weighted avg       0.61      0.63      0.58      6795

              precision    recall  f1-score   support

       False       0.63      0.90      0.74       998
        True       0.62      0.24      0.34       701

    accuracy                           0.63      1699
   macro avg       0.62      0.57      0.54      1699
weighted avg       0.62      0.63      0.58      1699



In [20]:
import pickle

with open("model.pkl", "wb") as f:
  pickle.dump(pipeline, f)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean", add_indicator=True)),
    ("model", GradientBoostingClassifier(n_estimators=200, max_depth=3))
])

y_train_class = y_train - X_train["phq9_score_start_3"] > 0
y_test_class = y_test - X_test["phq9_score_start_3"] > 0

pipeline.fit(X_train, y_train_class)
test_pred = pipeline.predict(X_test)

print(classification_report(y_train_class, pipeline.predict(X_train)))
print(classification_report(y_test_class, test_pred))

              precision    recall  f1-score   support

       False       0.71      0.95      0.81      4124
        True       0.83      0.40      0.54      2671

    accuracy                           0.73      6795
   macro avg       0.77      0.67      0.68      6795
weighted avg       0.76      0.73      0.71      6795

              precision    recall  f1-score   support

       False       0.62      0.86      0.72       998
        True       0.57      0.27      0.36       701

    accuracy                           0.61      1699
   macro avg       0.60      0.56      0.54      1699
weighted avg       0.60      0.61      0.57      1699



In [18]:
pd.DataFrame(pipeline.named_steps["model"].feature_importances_, index=pipeline.named_steps["imputer"].get_feature_names_out(X_train.columns), columns=["coef"]).sort_values(by="coef", ascending=False)

,coef
phq9_score_start_3,0.115453
sleep_ratio_asleep_in_bed_iqr,0.041715
steps_awake_mean,0.040781
sleep_ratio_asleep_in_bed_weekend_mean,0.036530
steps_rolling_6_max_recent,0.035324
...,...
missingindicator_educ,0.000000
missingindicator_bmi,0.000000
missingindicator_height,0.000000
missingindicator_birth,0.000000
